In [1]:
# Translated to .py by Anna Fernández
# Adapted to PandasBiogeme by Michel Bierlaire
# Sun Oct 21 23:36:04 2018

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
from biogeme.expressions import Beta, DefineVariable

df = pd.read_csv("optimaDataset.dat",'\t')
database = db.Database("optimaDataset",df)
pd.options.display.float_format = '{:.3g}'.format
globals().update(database.variables)

exclude =  ((Choice   ==  2 ) + (Choice == -1) + (  CostCarCHF   <=  0  )) >0
database.remove(exclude)

#Parameters to be estimated
# Arguments:
#   1  Name for report. Typically, the same as the variable
#   2  Starting value
#   3  Lower bound
#   4  Upper bound
#   5  0: estimate the parameter, 1: keep it fixed
ASC_CAR	 = Beta('ASC_CAR',0,None,None,1)
ASC_PT_SHORT	 = Beta('ASC_PT_SHORT',0,None,None,0)
ASC_PT_LONG	 = Beta('ASC_PT_LONG',0,None,None,0)
BETA_BURDEN	 = Beta('BETA_BURDEN',0,None,None,0)
BETA_COST_CAR	 = Beta('BETA_COST_CAR',0,None,None,0)
BETA_COST_PT	 = Beta('BETA_COST_PT',0,None,None,0)
BETA_TIME	 = Beta('BETA_TIME',0,None,None,0)

# Define here arithmetic expressions for name that are not directly
# available from the data

DistanceLessThan23  = DefineVariable('DistanceLessThan23', distance_km   <  23 ,database)
Burden  = DefineVariable('Burden',(  WalkingTimePT   +  WaitingTimePT   ) / (  NbTransf   +  1  ),database)

#Utilities
CAR = ASC_CAR + BETA_COST_CAR * CostCarCHF + BETA_TIME * TimeCar
PT = ASC_PT_SHORT * DistanceLessThan23 +  ASC_PT_LONG * (1-DistanceLessThan23) + BETA_COST_PT * MarginalCostPT + BETA_TIME * TimePT  + BETA_BURDEN * Burden
V = {1: CAR,0: PT}
av = {1: 1,0: 1}

# Logit model, with availability conditions
logprob = models.loglogit(V,av,Choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "binary_specific_optima"
results = biogeme.estimate()
# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)
print(f"Nbr of observations: {database.getNumberOfObservations()}")
print(f"LL(0) =    {results.data.initLogLike:.3f}")
print(f"LL(beta) = {results.data.logLike:.3f}")
print(f"rho bar square = {results.data.rhoBarSquare:.3g}")
print(f"Output file: {results.data.htmlFileName}")




                Value  Std err  t-test  p-value  Rob. Std err  Rob. t-test  \
ASC_PT_LONG     0.165    0.169   0.972    0.331          0.19        0.866   
ASC_PT_SHORT    -0.21    0.124    -1.7   0.0893         0.132         -1.6   
BETA_BURDEN   -0.0118  0.00451   -2.62  0.00887       0.00563         -2.1   
BETA_COST_CAR  -0.145   0.0145     -10        0        0.0257        -5.63   
BETA_COST_PT  -0.0676    0.008   -8.46        0        0.0149        -4.53   
BETA_TIME     -0.0122  0.00176   -6.95 3.69e-12       0.00315        -3.89   

               Rob. p-value  
ASC_PT_LONG           0.386  
ASC_PT_SHORT           0.11  
BETA_BURDEN           0.036  
BETA_COST_CAR      1.76e-08  
BETA_COST_PT       5.91e-06  
BETA_TIME          0.000102  
Nbr of observations: 1789
LL(0) =    -1240.040
LL(beta) = -918.008
rho bar square = 0.255
Output file: binary_specific_optima~00.html


/usr/local/lib/python3.7/site-packages/scipy/linalg/basic.py:1321: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  x, resids, rank, s = lstsq(a, b, cond=cond, check_finite=False)
